In [5]:
# import zakladnych kniznic
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree

# import tried
from src.python.country_utils import country_utils
from src.python.preprocess import preprocess as pre
%load country_utils.py

XGBoostLibraryNotFound: Cannot find XGBoost Library in the candidate path, did you install compilers and run build.sh in root path?
List of candidates:
C:\Users\stefi\AppData\Roaming\Python\Python36\site-packages\xgboost\xgboost.dll
C:\Users\stefi\AppData\Roaming\Python\Python36\site-packages\xgboost\../../lib/xgboost.dll
C:\Users\stefi\AppData\Roaming\Python\Python36\site-packages\xgboost\./lib/xgboost.dll
c:\users\stefi\appdata\local\programs\python\python36\xgboost\xgboost.dll
C:\Users\stefi\AppData\Roaming\Python\Python36\site-packages\xgboost\../../windows/x64/Release/xgboost.dll
C:\Users\stefi\AppData\Roaming\Python\Python36\site-packages\xgboost\./windows/x64/Release/xgboost.dll

In [ ]:
# nacitanie suborov
df_train = pd.read_csv("E:/Skola/PycharmProjects/nn_covid-19_predictions/data/models/c19-week-4/train.csv", parse_dates=['Date'])
df_test = pd.read_csv("E:/Skola/PycharmProjects/nn_covid-19_predictions/data/models/c19-week-4/test.csv", parse_dates=['Date'])

In [ ]:
# vypis hlaviciek a opisov datasetov
df_train.head()
df_train.describe()
df_train.info()

df_test.head()
df_train.describe()
df_test.info()

In [ ]:
# teraz berieme policko Date ako String, preto ho preformatujem na Datetime format
df_train['Date'] = pd.to_datetime(df_train['Date'], format='%Y-%m-%d')
df_test['Date'] = pd.to_datetime(df_test['Date'], format='%Y-%m-%d')

# a teraz ho uz vieme citat ako datum

In [ ]:
# vypis najstarsieho datumu a najnovsieho datumu z trenovacej mnoziny
train_date_min = df_train['Date'].min()
train_date_max = df_train['Date'].max()
print('Najstarší dátum z trénovacej množiny: {}'.format(train_date_min))
print('Najnovší dátum z trénovacej množiny: {}'.format(train_date_max))

In [ ]:
# to iste pre testovaciu mnozinu
test_date_min = df_test['Date'].min()
test_date_max = df_test['Date'].max()
print('Najstarší dátum z testovacej množiny: {}'.format(test_date_min))
print('Najnovší dátum z testovacej množiny: {}'.format(test_date_max))

# Predikcia nových nakazení

In [ ]:
df_train.head()
x_train = df_train[[]]

In [ ]:
# Pre predikciu novych pripadov vyuzijem Linearnu regresiu
# Pridala som import sklearn
for country in df_train['Country_Region'].unique():
    print('Robim model pre krajinu: {}'.format(country))
    df_train_country = df_train[df_train['Country_Region'] == country]
    df_test_country = df_test[df_test['Country_Region'] == country]
    
    # zaujima ma, ci ma krajina provinciu, ak ano -> vetva else
    if df_train_country['Province_State'].isna().unique().any():
        # indexujem krajiny v csv subore - od a do
        x_train = np.array(range(len(df_train_country))).reshape((-1, 1))
        # print(x_train)
        y_train = df_train_country['ConfirmedCases']
        # print(y_train)
        
        # ucenie siete
        model= Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression(fit_intercept=False))])
        model.fit(x_train, y_train)
        
        x_test = np.array(range(len(df_test_country))).reshape((-1, 1))
        prediction = model.predict(x_test)
        
        # negativne pripady nahradim nulou
        prediction[prediction < 0] = 0
        
        # Vramci ConfirmedCases pridam sa novy stlpec v dataframe df_test kam vlozim predikciu k danej krajine
        df_test.loc[df_test['Country_Region'] == country, 'ConfirmedCases'] = prediction
        # print(type(prediction))
        
    else:
        for state in df_train_country['Province_State'].unique():
            df_train_state = df_train_country[df_train_country['Province_State'] == state]
            df_test_state = df_test_country[df_test_country['Province_State'] == state]
            
            x_train = np.array(range(len(df_train_state))).reshape(-1, 1)
            y_train = df_train_state['ConfirmedCases']
            
            model= Pipeline([('poly', PolynomialFeatures(degree=2)), ('linear', LinearRegression(fit_intercept=False))])
            model.fit(x_train, y_train)
            
            x_test = np.array(range(len(df_test_state))).reshape((-1, 1))
            prediction = model.predict(x_test)
            
            prediction[prediction < 0] = 0
            
            # Vramci ConfirmedCases pridam sa novy stlpec v dataframe df_test kam vlozim predikciu k danej krajine
            df_test.loc[(df_test['Country_Region'] == country) & (df_test['Province_State'] == state), 'ConfirmedCases'] = prediction

## Vloženie naučených dát do *submission.csv*

In [ ]:
# len vlozim data do pripraveneho csv suboru
df_submit = pd.read_csv('E:/Skola/PycharmProjects/nn_covid-19_predictions/data/models/c19-week-4/submission.csv')
df_submit['ConfirmedCases'] = df_test['ConfirmedCases'].astype('int')
df_submit.to_csv('E:/Skola/PycharmProjects/nn_covid-19_predictions/data/models/c19-week-4/submission.csv', index=False)

In [ ]:
df_test.head()

In [ ]:
df_train.info()